# Using PCSS QAPI for ORCA-PT-1 

## Logging in

In [3]:
from pcss_qapi import AuthorizationService
AuthorizationService.login()

ℹ️ Using cached tokens. If you want to change the account use AuthorizationService.logout() first.


## Getting a provider

In [4]:
from pcss_qapi.orca import OrcaProvider

provider = OrcaProvider()

## Getting a backend

A list of available backends can be fetched from the provider.

In [5]:
provider.available_backends(simulators=True)

['single_loop_simulator', 'multi_loop_simulator']

In [6]:
provider.available_backends(simulators=False)

['ORCA-PT-1-A', 'ORCA-PT-1-B']

A backend can either be specified by name or automatically fetched based on the smallest queue length (real devices only).

In [7]:
backend = provider.get_backend('single_loop_simulator')
backend

OrcaBackend[single_loop_simulator]

In [8]:
backend = provider.least_busy()
backend

OrcaBackend[ORCA-PT-1-A]

## Using the ORCA-PT-1 quantum computer

Each backend represents a single quantum computer (or simulator) and provides a few options of interacting with the device:
- A simple TBI (Time Bin Inferometer) class for sampling
- A PyTorch compatible layer for integrating the PT-1 into neural networks
- A Binary Bosonic Solver class for solving combinatorial optimization problems

### TBI

In [9]:
backend = provider.get_backend("ORCA-PT-1-A")

tbi = backend.get_tbi(n_loops=1,loop_lengths=[1])
tbi.sample(
    input_state=[0,1,0],
    theta_list=[1,2],
    n_samples=200,
    n_tiling=1
    )

{(0, 0, 1): 39, (0, 1, 0): 7, (1, 0, 0): 154}

### PTLayer (PyTorch compatible)

In [10]:
import torch

ptlayer = backend.get_ptlayer(in_features=2)
ptlayer(torch.tensor([[1.0,-1.0]],dtype=torch.float32))


tensor([[0.7225, 0.0975, 0.1800]])

### Binary Bosonic Solver

In [11]:
bbs = backend.get_bbs(2, lambda x: -x[0])

bbs.train(updates=2,print_frequency=1)


Training loop 1: loss is -0.46
Training loop 2: loss is -0.53
